# Atualização dos dados

Como dito anteriormente, optamos por não utilizar os dados direto da API para evitar exceder o limite de requisições. Assim, vamos utilizar os dados do ergast para montar o histórico e baixar a partir de 2025 em diante utilizando a API. Como já escolhemos os dados que iremos utilizar vamos baixar apenas os dados necessários.

## Baixando dados do Ergast

In [69]:
from utils.ergast import download_ergast, load_ergast

download_ergast()

data = load_ergast()

drivers = data['drivers']
results = data['results']
driver_standings = data['driver_standings']
constructors = data['constructors']
constructor_results = data['constructor_results']
constructor_standings = data['constructor_standings']
races = data['races']
circuits = data['circuits']
lap_times = data['lap_times']
pit_stops = data['pit_stops']
qualifying = data['qualifying']
seasons = data['seasons']
sprint_results = data['sprint_results']
status = data['status']

Data already downloaded.


## Interface com API

In [ ]:
import requests
import pandas as pd



jolpica = Jolpica()

## Construindo o banco de dados

### Drivers (OK)

- `drivers`: tabela com os pilotos
    - `driverId`: id do piloto
    - `driverIdErgast`: id do piloto no ergast
    - `code`: código do piloto
    - `givenName`: nome do piloto
    - `familyName`: sobrenome do piloto
    - `dateOfBirth`: data de nascimento do piloto
    - `nationality`: nacionalidade do piloto
    - `url`: url do piloto

In [ ]:
drivers_old_map_columns = {
    'driverId': 'driverIdErgast',
    'code': 'code',
    'forename': 'givenName',
    'surname': 'familyName',
    'dob': 'dateOfBirth',
    'nationality': 'nationality',
    'url': 'url',
}

e_drivers = drivers[list(drivers_old_map_columns.keys())].rename(columns=drivers_old_map_columns)

j_drivers = pd.DataFrame()
offset = 0
total = 1
while j_drivers.shape[0] < total:
    print(f"Drivers: {j_drivers.shape[0]} / {total}")
    temp = jolpica.drivers(offset=offset)
    j_drivers = pd.concat([j_drivers, pd.DataFrame(temp['MRData']['DriverTable']['Drivers'])], ignore_index=True)
    total = int(temp['MRData']['total'])
    offset += 100

j_drivers = j_drivers[['driverId', 'code', 'givenName', 'familyName', 'dateOfBirth', 'nationality', 'url']]

e_drivers['name'] = e_drivers['givenName'] + ' ' + e_drivers['familyName']
j_drivers['name'] = j_drivers['givenName'] + ' ' + j_drivers['familyName']

c_drivers = pd.merge(
    e_drivers[['driverIdErgast', 'name']],
    j_drivers,
    how='outer',
    on='name',
)
c_drivers = c_drivers.drop(columns=['name'])
c_drivers

Drivers: 0 / 1
Drivers: 100 / 864
Drivers: 200 / 864
Drivers: 300 / 864
Drivers: 400 / 864
Drivers: 500 / 864
Drivers: 600 / 864
Drivers: 700 / 864
Drivers: 800 / 864


,driverIdErgast,driverId,code,givenName,familyName,dateOfBirth,nationality,url
0,750.0,brudes,NaN,Adolf,Brudes,1899-10-15,German,http://en.wikipedia.org/wiki/Adolf_Brudes
1,698.0,cruz,NaN,Adolfo,Cruz,1923-06-28,Argentine,http://en.wikipedia.org/wiki/Adolfo_Schewelm_Cruz
2,16.0,sutil,SUT,Adrian,Sutil,1983-01-11,German,http://en.wikipedia.org/wiki/Adrian_Sutil
3,167.0,campos,NaN,Adrián,Campos,1960-06-17,Spanish,http://en.wikipedia.org/wiki/Adri%C3%A1n_Campos
4,88.0,suzuki,NaN,Aguri,Suzuki,1960-09-08,Japanese,http://en.wikipedia.org/wiki/Aguri_Suzuki
...,...,...,...,...,...,...,...,...
859,47.0,baumgartner,NaN,Zsolt,Baumgartner,1981-01-01,Hungarian,http://en.wikipedia.org/wiki/Zsolt_Baumgartner
860,625.0,bayol,NaN,Élie,Bayol,1914-02-28,French,http://en.wikipedia.org/wiki/%C3%89lie_Bayol
861,103.0,bernard,NaN,Éric,Bernard,1964-08-24,French,http://en.wikipedia.org/wiki/%C3%89ric_Bernard
862,100.0,comas,NaN,Érik,Comas,1963-09-28,French,http://en.wikipedia.org/wiki/%C3%89rik_Comas


### Constructors (OK)

- `constructors`: tabela com as equipes
    - `constructorId`: id da equipe
    - `constructorIdErgast`: id da equipe no ergast
    - `name`: nome da equipe
    - `nationality`: nacionalidade da equipe
    - `url`: url da equipe

In [ ]:
constructors_old_map_columns = {
    'constructorId': 'constructorIdErgast',
    'name': 'name',
}

e_constructors = constructors[list(constructors_old_map_columns.keys())].rename(columns=constructors_old_map_columns)

# listando constructors
j_constructors = pd.DataFrame()
offset = 0
total = 1
while j_constructors.shape[0] < total:
    print(f"constructors: {j_constructors.shape[0]} / {total}")
    temp = jolpica.constructors(offset=offset)
    j_constructors = pd.concat([j_constructors, pd.DataFrame(temp['MRData']['ConstructorTable']['Constructors'])], ignore_index=True)
    total = int(temp['MRData']['total'])
    offset += 100

c_constructors = pd.merge(e_constructors, j_constructors, how='outer', on='name')
c_constructors

constructors: 0 / 1
constructors: 100 / 212
constructors: 200 / 212


,constructorIdErgast,name,constructorId,url,nationality
0,147,AFM,afm,http://en.wikipedia.org/wiki/Alex_von_Falkenha...,German
1,39,AGS,ags,http://en.wikipedia.org/wiki/Automobiles_Gonfa...,French
2,54,ATS,ats,http://en.wikipedia.org/wiki/ATS_(wheels),Italian
3,161,Adams,adams,http://en.wikipedia.org/wiki/Adams_(constructor),American
4,51,Alfa Romeo,alfa,http://en.wikipedia.org/wiki/Alfa_Romeo_in_For...,Swiss
...,...,...,...,...,...
207,107,Watson,watson,http://en.wikipedia.org/wiki/A.J._Watson,American
208,160,Wetteroth,wetteroth,http://en.wikipedia.org/wiki/Wetteroth,American
209,3,Williams,williams,http://en.wikipedia.org/wiki/Williams_Grand_Pr...,British
210,59,Wolf,wolf,http://en.wikipedia.org/wiki/Walter_Wolf_Racing,Canadian


### Circuits (OK)

- `circuits`: tabela com os circuitos
    - `circuitId`: id do circuito
    - `circuitIdErgast`: id do circuito no ergast
    - `circuitName`: nome do circuito
    - `locality`: localização do circuito
    - `lat`: latitude do circuito
    - `long`: longitude do circuito
    - `country`: país do circuito
    - `url`: url do circuito

In [95]:
circutis_old_map_columns = {
    'circuitId': 'circuitIdErgast',
    'name': 'circuitName',
    'location': 'locality',
    'country': 'country',
    'lat': 'lat',
    'lng': 'long',
    'url': 'url',
}

e_circuits = circuits[list(circutis_old_map_columns.keys())].rename(columns=circutis_old_map_columns)

j_circuits = pd.DataFrame()
offset = 0
total = 1
while j_circuits.shape[0] < total:
    print(f"circuits: {j_circuits.shape[0]} / {total}")
    temp = jolpica.circuits(offset=offset)
    j_circuits = pd.concat([j_circuits, pd.DataFrame(temp['MRData']['CircuitTable']['Circuits'])], ignore_index=True)
    total = int(temp['MRData']['total'])
    offset += 100

c_circuits = pd.merge(
    e_circuits,
    j_circuits[['circuitName', 'circuitId']],
    on='circuitName',
    how='outer',
)

c_circuits

circuits: 0 / 1


,circuitIdErgast,circuitName,locality,country,lat,long,url,circuitId
0,61,AVUS,Berlin,Germany,52.4806,13.251400,http://en.wikipedia.org/wiki/AVUS,avus
1,29,Adelaide Street Circuit,Adelaide,Australia,-34.9272,138.617000,http://en.wikipedia.org/wiki/Adelaide_Street_C...,adelaide
2,64,Ain Diab,Casablanca,Morocco,33.5786,-7.687500,http://en.wikipedia.org/wiki/Ain-Diab_Circuit,ain-diab
3,58,Aintree,Liverpool,UK,53.4769,-2.940560,http://en.wikipedia.org/wiki/Aintree_Motor_Rac...,aintree
4,1,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.8497,144.968000,http://en.wikipedia.org/wiki/Melbourne_Grand_P...,albert_park
...,...,...,...,...,...,...,...,...
72,12,Valencia Street Circuit,Valencia,Spain,39.4589,-0.331667,http://en.wikipedia.org/wiki/Valencia_Street_C...,valencia
73,46,Watkins Glen,New York State,USA,42.3369,-76.927200,http://en.wikipedia.org/wiki/Watkins_Glen_Inte...,watkins_glen
74,24,Yas Marina Circuit,Abu Dhabi,UAE,24.4672,54.603100,http://en.wikipedia.org/wiki/Yas_Marina_Circuit,yas_marina
75,57,Zeltweg,Styria,Austria,47.2039,14.747800,http://en.wikipedia.org/wiki/Zeltweg_Airfield,zeltweg


### Races (OK)

- `races`: tabela com as corridas
    - `season`: ano da corrida
    - `round`: número da corrida
    - `raceName`: nome da corrida
    - `date`: data da corrida
    - `circuitId`: id do circuito
    - `url`: url da corrida

In [165]:
import datetime

races_old_map_columns = {
    'raceId': 'raceIdErgast',
    'year': 'season',
    'round': 'round',
    'circuitId': 'circuitId',
    'name': 'raceName',
    'date': 'date',
    'url': 'url',
}
e_races = races[list(races_old_map_columns.keys())].rename(columns=races_old_map_columns)

e_races['circuitId'] = e_races['circuitId'].map(c_circuits.set_index('circuitIdErgast')['circuitId'].to_dict())

last_season = e_races[e_races.date == e_races.date.max()].season.iloc[0].item()

j_races = pd.DataFrame()
for season in range(last_season, datetime.datetime.today().year + 1):
    print(season)
    temp = jolpica.races(season=season)
    j_races = pd.concat([
        j_races,
        pd.DataFrame(temp['MRData']['RaceTable']['Races'])
    ], ignore_index=True)

j_races['circuitId'] = j_races['Circuit'].apply(lambda x: x['circuitId'])

j_races = j_races[['season', 'round', 'circuitId', 'raceName', 'date', 'url']]
j_races['season'] = j_races['season'].astype(int)
j_races['round'] = j_races['round'].astype(int)

c_races = pd.concat([e_races, j_races], ignore_index=True).drop_duplicates(subset=['season', 'round'], keep='first').sort_values(by=['season', 'round'])

c_races

2024
2025


,raceIdErgast,season,round,circuitId,raceName,date,url
832,833.0,1950,1,silverstone,British Grand Prix,1950-05-13,http://en.wikipedia.org/wiki/1950_British_Gran...
833,834.0,1950,2,monaco,Monaco Grand Prix,1950-05-21,http://en.wikipedia.org/wiki/1950_Monaco_Grand...
834,835.0,1950,3,indianapolis,Indianapolis 500,1950-05-30,http://en.wikipedia.org/wiki/1950_Indianapolis...
835,836.0,1950,4,bremgarten,Swiss Grand Prix,1950-06-04,http://en.wikipedia.org/wiki/1950_Swiss_Grand_...
836,837.0,1950,5,spa,Belgian Grand Prix,1950-06-18,http://en.wikipedia.org/wiki/1950_Belgian_Gran...
...,...,...,...,...,...,...,...
1168,NaN,2025,20,rodriguez,Mexico City Grand Prix,2025-10-26,https://en.wikipedia.org/wiki/2025_Mexico_City...
1169,NaN,2025,21,interlagos,São Paulo Grand Prix,2025-11-09,https://en.wikipedia.org/wiki/2025_S%C3%A3o_Pa...
1170,NaN,2025,22,vegas,Las Vegas Grand Prix,2025-11-23,https://en.wikipedia.org/wiki/2025_Las_Vegas_G...
1171,NaN,2025,23,losail,Qatar Grand Prix,2025-11-30,https://en.wikipedia.org/wiki/2025_Qatar_Grand...


### Constructor Standings (OK)

- `constructor_standings`: tabela com a classificação das equipes
    - `season`: ano da corrida
    - `round`: número da corrida
    - `position`: posição da equipe na corrida
    - `points`: pontos da equipe na corrida
    - `wins`: vitórias da equipe na corrida
    - `constructorId`: id da equipe

In [199]:
constructor_standings_old_map_columns = {
    'raceId': 'raceId',
    'constructorId': 'constructorId',
    'points': 'points',
    'position': 'position',
    'wins': 'wins',
}

e_constructor_standings = constructor_standings[list(constructor_standings_old_map_columns.keys())].rename(columns=constructor_standings_old_map_columns)

e_constructor_standings = pd.merge(
    e_constructor_standings.assign(raceId=e_constructor_standings.raceId.astype(float)),
    c_races[['raceIdErgast', 'season', 'round']].rename(columns={'raceIdErgast': 'raceId'}),
    how='left',
    on='raceId',
).drop(columns=['raceId'])

e_constructor_standings['constructorId'] = e_constructor_standings['constructorId'].map(c_constructors.set_index('constructorIdErgast')['constructorId'].to_dict())

from tqdm import tqdm

missing_season_races = set([(x[0].item(), x[1].item()) for x in c_races[['season', 'round']].to_numpy()]) \
    - set([(x[0].item(), x[1].item()) for x in e_constructor_standings[['season', 'round']].to_numpy()])

last_season = max(missing_season_races)[0]
last_round_last_season = int(jolpica.constructor_standing(last_season)['MRData']['StandingsTable']['round'])
missing_season_races = [x for x in missing_season_races if x <= (last_season, last_round_last_season)]
missing_season_races = [x for x in missing_season_races if x[0] > 2000]

j_constructors_standing = pd.DataFrame()
for season, round in tqdm(missing_season_races):
    temp = jolpica.constructor_standing(season=season, round=round)
    temp_df = pd.DataFrame(temp['MRData']['StandingsTable']['StandingsLists'][0]['ConstructorStandings'])
    temp_df['season'] = season
    temp_df['round'] = round
    temp_df['constructorId'] = temp_df['Constructor'].apply(lambda x: x['constructorId'])
    j_constructors_standing = pd.concat([
        j_constructors_standing,
        temp_df
    ], ignore_index=True)

j_constructors_standing = j_constructors_standing[['season', 'round', 'constructorId', 'points', 'position', 'wins']]


c_constructor_standings = pd.concat([
    e_constructor_standings,
    j_constructors_standing
], ignore_index=True).drop_duplicates(subset=['season', 'round', 'constructorId'], keep='first').sort_values(by=['season', 'round'])

c_constructor_standings

100%|██████████| 4/4 [00:04<00:00,  1.00s/it]


,constructorId,points,position,wins,season,round
9387,maserati,3.0,3,0,1958,1
9388,ferrari,6.0,2,0,1958,1
9389,cooper,8.0,1,1,1958,1
9375,osca,0.0,8,0,1958,2
9376,connaught,0.0,7,0,1958,2
...,...,...,...,...,...,...
13406,williams,19,6,0,2025,4
13407,aston_martin,10,7,0,2025,4
13408,rb,7,8,0,2025,4
13409,alpine,6,9,0,2025,4


### Driver Standings (OK)

- `driver_standings`: tabela com a classificação dos pilotos
    - `season`: ano da corrida
    - `round`: número da corrida
    - `position`: posição do piloto na corrida
    - `points`: pontos do piloto na corrida
    - `wins`: vitórias do piloto na corrida
    - `driverId`: id do piloto
    - `constructorId`: id da equipe

In [259]:
driver_standings_old_map_columns = {
    'driverId': 'driverId',
    'points': 'points',
    'position': 'position',
    'wins': 'wins',

    'raceId': 'raceId',
}

e_driver_standings = driver_standings[list(driver_standings_old_map_columns.keys())].rename(columns=driver_standings_old_map_columns)

e_driver_standings['driverId'] = e_driver_standings['driverId'].astype(float).map(c_drivers.dropna(subset=['driverIdErgast']).set_index('driverIdErgast')['driverId'].to_dict())

e_driver_standings = pd.merge(
    e_driver_standings,
    c_races[['raceIdErgast', 'season', 'round']].rename(columns={'raceIdErgast': 'raceId'}),
    how='left',
).drop(columns=['raceId'])

missing_season_races = set([(x[0].item(), x[1].item()) for x in c_constructor_standings[['season', 'round']].to_numpy()])

seasons = list(set([x[0] for x in missing_season_races]))

drivers_constructors_seasons = pd.DataFrame()
for season in tqdm(seasons):
    temp = jolpica.driver_standing(season=season)
    temp_df = pd.DataFrame(temp['MRData']['StandingsTable']['StandingsLists'][0]['DriverStandings'])
    temp_df['season'] = season
    drivers_constructors_seasons = pd.concat([
        drivers_constructors_seasons,
        temp_df
    ], ignore_index=True)

drivers_constructors_seasons['constructorId'] = drivers_constructors_seasons['Constructors'].apply(lambda x: x[0]['constructorId'])
drivers_constructors_seasons['driverId'] = drivers_constructors_seasons['Driver'].apply(lambda x: x['driverId'])

e_driver_standings = pd.merge(
    e_driver_standings,
    drivers_constructors_seasons[['driverId', 'constructorId', 'season']],
    on=['driverId', 'season'],
    how='left'
)

missing_season_races = set([(x[0].item(), x[1].item()) for x in c_races[['season', 'round']].to_numpy()]) \
    - set([(x[0].item(), x[1].item()) for x in e_driver_standings[['season', 'round']].to_numpy()])

last_season = max(missing_season_races)[0]
last_round_last_season = int(jolpica.driver_standing(last_season)['MRData']['StandingsTable']['round'])
missing_season_races = [x for x in missing_season_races if x <= (last_season, last_round_last_season)]
missing_season_races = [x for x in missing_season_races if x[0] > 2000]

j_drivers_standing = pd.DataFrame()
for season, round in tqdm(missing_season_races):
    temp = jolpica.driver_standing(season=season, round=round)
    temp_df = pd.DataFrame(temp['MRData']['StandingsTable']['StandingsLists'][0]['DriverStandings'])
    temp_df['season'] = season
    temp_df['round'] = round
    temp_df['driverId'] = temp_df['Driver'].apply(lambda x: x['driverId'])
    temp_df['constructorId'] = temp_df['Constructors'].apply(lambda x: x[0]['constructorId'])
    j_drivers_standing = pd.concat([
        j_drivers_standing,
        temp_df
    ], ignore_index=True)

j_drivers_standing = j_drivers_standing[['season', 'round', 'driverId', 'constructorId', 'points', 'position', 'wins']]

c_driver_standings = pd.concat([
    e_driver_standings,
    j_drivers_standing
], ignore_index=True).drop_duplicates(subset=['season', 'round', 'driverId'], keep='first').sort_values(by=['season', 'round'])

100%|██████████| 4/4 [00:03<00:00,  1.25it/s]
